# Multi Agent orchestrator using Azure AI Agentic Service

**Problem Statment**: AI agents are autonomous software entities designed to perform tasks, make decisions, and interact with environments using artificial intelligence, machine learning, natural language processing, and reinforcement learning. However, every custom AI agent written today needs lifecycle management, that includes packaging as containers, deployment, scaling, allocation of right resources etc.
As the number of agents grow in your ecosystem it becomes tedious to manage the environment. 

**Azure AI Agent Service** is a fully managed service designed to empower developers to securely build, deploy, and scale high-quality, and extensible AI agents without needing to manage the underlying compute and storage resources. We can use Azure AI Agent Service to create and run an agent in just a few lines of code. You can also manage complex workflows with **AutoGen** and **Semantic Kernel**.


In this example we will summarize a research paper into LinkedIn post using Azure Agentic Service, Bing & LLMs. 



In [35]:
## Project PIP requirements
# %pip install -q azure-ai-projects azure-identity azure-ai-ml azure-search-documents tika "autogen-agentchat" "autogen-ext[openai]"
# %pip install -q -U "autogen-agentchat" "autogen-ext[openai]"

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.models import UserMessage


# Load environment variables from .env file
load_dotenv()

# Research paper path
research_paper_path =  "https://arxiv.org/pdf/2503.05142"

Before proceeding make sure you have created an Azure AI Agentic Service project. 

In [38]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

conn_list = project_client.connections._list_connections()["value"]
conn_id = ""

# Search in the metadata field of each connection in the list for the azure_ai_search type and get the id value to establish the variable
for conn in conn_list:
    metadata = conn["properties"].get("metadata", {})
    if metadata.get("type", "").upper() == "AZURE_AI_SEARCH":
        conn_id = conn["id"]
        break
    
print(f"Connection ID: {conn_id}")

Connection ID: /subscriptions/6a01260f-39d6-415f-a6c9-cf4fd479cbec/resourceGroups/rg-vism-7896_ai/providers/Microsoft.MachineLearningServices/workspaces/agentic-service/connections/vectorsearchagentic


In [39]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    api_version="2024-05-01-preview",
    model = "gpt-4o-mini",
    azure_endpoint=os.environ["AOAI_ENDPOINT"], # Azure OpenAI endpoint.
    api_key=os.environ["AOAI_KEY"], # For key-based authentication.
)

# Test the Azure OpenAI model client
result = await az_model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=8) cached=False logprobs=None thought=None


**Pre-requisites**
1. Create Azure AI Search and configure as connector to the agent. [Link](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=pythonsdk%2Cpython&pivots=code-examples#setup-create-an-agent-that-can-use-an-existing-azure-ai-search-index)

### Download and ingest the file to vector store

In [40]:
import requests
import uuid
from tika import parser 
os.makedirs("./data", exist_ok=True)
file_path = "./data/research_paper.pdf"
if not os.path.exists(file_path):
    print("Downloading research paper...")
    response = requests.get(research_paper_path)
    with open(file_path, "wb") as f:
       f.write(response.content)
else:
    print("Research paper already downloaded.")
raw = parser.from_file(file_path)
content = raw['content']

Research paper already downloaded.


### Create Bing Search Agent

In [41]:
# create environment variable
os.environ["BING_CONNECTION_NAME "] = "bing-grounding"

bing_connection = project_client.connections.get(
    connection_name="binggrounding"
)

In [42]:
from azure.ai.projects.models import BingGroundingTool

async def web_ai_agent(query: str) -> str:
    
    conn_id = bing_connection.id
    
    bing = BingGroundingTool(connection_id=conn_id)
    
    project_client = AIProjectClient.from_connection_string(credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"])
    
    
    with project_client:
        
        agent = project_client.agents.create_agent(
            model="gpt-4o",
            name="bing-search-assistant",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]

In [43]:
bing_search_agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="Use tools to solve tasks.",
)

print("Bing Search Agent created")


Bing Search Agent created


In [44]:
async def save_blog_agent(blog_content: str) -> str:
    
    print("This is Code Interpreter for Azure AI Agent Service .......")
    
    project_client = AIProjectClient.from_connection_string(credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"])
    
    code_interpreter = CodeInterpreterTool()
    
    agent = project_client.agents.create_agent(
            model="gpt-4o",
            name="blog-save-agent",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
            # tool_resources=code_interpreter.resources,
    )

    thread = project_client.agents.create_thread()

    message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="""
        
                    You are my Python programming assistant. Generate code,save """+ blog_content +
                    
                """    
                    and execute it according to the following requirements

                    1. Save blog content to blog-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    )
    
    # create and execute a run
    
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")
        # print the messages from the agent

    messages = project_client.agents.list_messages(thread_id=thread.id)

    print(f"Messages: {messages}")

    # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_role("assistant")
    
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")
        
        
    project_client.agents.delete_agent(agent.id)
    
    print("Deleted agent")

    return "Saved"

In [45]:
save_blog_content_agent = AssistantAgent(
    name="save_post_content_agent",
    model_client=az_model_client,
    tools=[save_blog_agent],
    system_message="""
        Save post content. Respond with 'Saved' to when your post are saved.
    """
)

In [46]:
write_agent = AssistantAgent(
    name="write_agent",
    model_client=az_model_client,
    system_message="""
        You are a linked in post writer, please help me write a linked post based on research paper and bing search content."
    """
)

### Run Orchestration

In [47]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(max_messages=5)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination
reflection_team = RoundRobinGroupChat([bing_search_agent, write_agent,save_blog_content_agent], termination_condition=termination)

In [48]:
await Console(
    reflection_team.run_stream(task=f"""
                    I'm writing a linked in post about a research paper. The content of the research paper is {content}.
                    
                    Extract the following information from the research paper:
                    
                    - Problem/Research Question: What is the research trying to address or investigate? 
                    - Methods/Approach: How did the researchers conduct their study? 
                    - Results/Findings: What did the research uncover? 
                    - Conclusions/Implications: What are the main takeaways and significance of the research? 
                    - Limitations and Future Directions: What
                    
                    Generate a high-engagement Linked In post about the challenges and solutions in the research paper.
                    
                    Extract keywords and use bing search to explain the keywords in the research paper.                    
                    The tone should be authoritative yet engaging. 
                    Follow the Hook → Context → Insights → Engagement Trigger structure. End with a question to spark discussion. Add 3-5 relevant hashtags.
    """)
)  

---------- user ----------

                    I'm writing a linked in post about a research paper. The content of the research paper is 




































Published as a conference paper at ICLR 2025

ROCKETEVAL: EFFICIENT AUTOMATED LLM
EVALUATION VIA GRADING CHECKLIST

Tianjun Wei∗† 12, Wei Wen∗ 2, Ruizhi Qiao‡2, Xing Sun 2, Jianghong Ma‡ 3

1 City University of Hong Kong, 2 Tencent Youtu Lab, 3 Harbin Institute of Technology Shenzhen.
tjwei2-c@my.cityu.edu.hk {jawnrwen,ruizhiqiao,winfredsun}@tencent.com
majianghong@hit.edu.cn

ABSTRACT

Evaluating large language models (LLMs) in diverse and challenging scenarios
is essential to align them with human preferences. To mitigate the prohibitive
costs associated with human evaluations, utilizing a powerful LLM as a judge
has emerged as a favored approach. Nevertheless, this methodology encounters
several challenges, including substantial expenses, concerns regarding privacy
and security, and reproducibility. In this pap

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='\n                    I\'m writing a linked in post about a research paper. The content of the research paper is \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPublished as a conference paper at ICLR 2025\n\nROCKETEVAL: EFFICIENT AUTOMATED LLM\nEVALUATION VIA GRADING CHECKLIST\n\nTianjun Wei∗† 12, Wei Wen∗ 2, Ruizhi Qiao‡2, Xing Sun 2, Jianghong Ma‡ 3\n\n1 City University of Hong Kong, 2 Tencent Youtu Lab, 3 Harbin Institute of Technology Shenzhen.\ntjwei2-c@my.cityu.edu.hk {jawnrwen,ruizhiqiao,winfredsun}@tencent.com\nmajianghong@hit.edu.cn\n\nABSTRACT\n\nEvaluating large language models (LLMs) in diverse and challenging scenarios\nis essential to align them with human preferences. To mitigate the prohibitive\ncosts associated with human evaluations, utilizing a powerful LLM as a judge\nhas emerged as a favored approach. Nevertheless, this methodology encounters\nseveral